In [28]:
import numpy as np
import random
import gurobipy as gp
import time

In [29]:
def generate_instances(n, p, capacity_mean, capacity_stddev):
    # Generate positions for nodes
    nodes = np.random.rand(n, 2)

    # Generate capacities for clusters
    capacities = np.random.normal(capacity_mean, capacity_stddev, p)

    return nodes, capacities

In [30]:
def generate_weights(n, weight_mean, weight_stddev):
    # Generate weights for nodes
    weights = np.random.normal(weight_mean, weight_stddev, n)

    return weights

In [31]:
def solve_ccp(n, p, nodes, capacities, weights):
    # Create a Gurobi model
    model = gp.Model()

    # Add decision variables
    x = {}
    y = {}
    for i in range(n):
        for j in range(p):
            x[i,j] = model.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))
        y[i] = model.addVar(vtype=gp.GRB.BINARY, name="y_{}".format(i))

    # Update the model
    model.update()

    # Set objective function
    obj = gp.quicksum(gp.quicksum(np.sqrt((nodes[i][0]-nodes[j][0])**2 + (nodes[i][1]-nodes[j][1])**2)*x[i,j] for j in range(n)) + weights[i]*(1-gp.quicksum(x[i,j] for j in range(p))) for i in range(n))

    model.setObjective(obj, gp.GRB.MINIMIZE)

    # Add constraints
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,j] for j in range(p)) == 1)

    for j in range(p):
        model.addConstr(gp.quicksum(weights[i]*x[i,j] for i in range(n)) <= capacities[j])

    for i in range(n):
        for j in range(p):
            model.addConstr(x[i,j] <= y[j])

    model.addConstr(gp.quicksum(y[j] for j in range(p)) <= p)

    # Optimize the model
    model.optimize()

    return model

In [33]:
def solve_with_heuristic(nodes, weights, capacities, p):
    n = len(nodes)

    # Initialize the clusters and the cluster weights
    clusters = [[nodes[i]] for i in range(n)]
    cluster_weights = [weights[i] for i in range(n)]

    # Sort the nodes by weight in decreasing order
    node_indices = np.argsort(weights)[::-1]

    # Iterate over the nodes
    for i in range(n):
        node = node_indices[i]

        # Find the cluster with the minimum weight
        min_cluster = np.argmin(cluster_weights)

        # If the weight of the cluster is less than p, merge the node with the cluster
        if cluster_weights[min_cluster] < p:
            clusters[min_cluster].append(node)
            cluster_weights[min_cluster] += weights[node]

    return clusters, cluster_weights

In [34]:
def evaluate_objective_function(model, nodes, clusters, cluster_weights):
    obj_value = 0
    for i in range(len(clusters)):
        cluster = clusters[i]
        weight = cluster_weights[i]
        for j in range(len(cluster)):
            for k in range(j+1, len(cluster)):
                node_j = cluster[j]
                node_k = cluster[k]
                obj_value += model(node_j, node_k, weight)
    return obj_value

In [35]:
# Study the stability and variability of the solutions
def study_stability(model, n, p, nodes, capacities, weights, num_trials):
    stability = []
    for _ in range(num_trials):
        # Generate random realizations of the weights
        random_weights = generate_weights(n, weights[0], weights[1])

        # Solve the model using the heuristic algorithm
        clusters, cluster_weights = solve_with_heuristic(nodes, random_weights, capacities, p)

        # Evaluate the performance of the solution using the objective function
        obj_value = evaluate_objective_function(model, nodes, clusters, cluster_weights)

        # Append the results to the stability list
        stability.append(obj_value)

    # Calculate the mean and standard deviation of the stability list
    mean = np.mean(stability)
    stddev = np.std(stability)

    return mean, stddev

In [36]:
def kmeans(data, k, max_iterations=100):
    # Initialize cluster centers
    cluster_centers = data[np.random.choice(data.shape[0], k, replace=False)]

    for iteration in range(max_iterations):
        # Calculate distances from each data point to cluster centers
        distances = np.array([np.linalg.norm(data - center, axis=1) for center in cluster_centers])

        # Assign each data point to the closest cluster center
        cluster_assignments = np.argmin(distances, axis=0)

        # Calculate new cluster centers
        new_cluster_centers = np.array([np.mean(data[cluster_assignments == i], axis=0) for i in range(k)])

        # Check for convergence
        if np.allclose(cluster_centers, new_cluster_centers):
            break

        cluster_centers = new_cluster_centers

    return cluster_assignments, cluster_centers

In [37]:
# Define the problem parameters
n = 100 # Number of nodes
p = 20 # Number of clusters
capacity_mean = 100 # Mean of capacities of clusters
capacity_stddev = 20 # Standard deviation of capacities of clusters
weight_mean = 50 # Mean of the weights of nodes
weight_stddev = 20 # Standard deviation of the weights of nodes

# Generate problem instances
nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
weights = generate_weights(n, weight_mean, weight_stddev)

KeyError: (0, 20)

In [ ]:
# Solve the CCP using Gurobi
model = solve_ccp(n, p, nodes, capacities, weights)

# Study the stability of the solution
num_trials = 50 # Number of trials for stability study
mean, stddev = study_stability(model, n, p, nodes, capacities, (weight_mean, weight_stddev), num_trials)

print("Mean of the stability: ", mean)
print("Standard deviation of the stability: ", stddev)